### Infraestrutura

importando as libs

In [ ]:
import subprocess
import sys

In [ ]:
# Lista de pacotes a serem instalados
packages = [
    "matplotlib",
    "numpy",
    "pandas",
    "scipy",
    "seaborn",
    "scikit-learn",
    "pyautogui",
    "pillow",
    "kmodes"
]

# Função para instalar pacotes com tentativa de pip e conda
def install_package(package):
    try:
        # Tentar instalar com pip
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} instalado com sucesso via pip!")
    except subprocess.CalledProcessError:
        print(f"Erro ao instalar o pacote {package} via pip. Tentando alternativas...")
        if package == "scikit-learn-extra":
            # Tentativa de instalação via conda para scikit-learn-extra
            try:
                print("Tentando instalar scikit-learn-extra usando conda...")
                subprocess.check_call(["conda", "install", "-c", "conda-forge", "scikit-learn-extra", "-y"])
                print("scikit-learn-extra instalado com sucesso via conda!")
            except subprocess.CalledProcessError:
                print(f"Erro ao instalar {package} com conda. Verifique seu ambiente Conda.")
        else:
            print(f"Não há alternativa para o pacote {package}. Instale manualmente se necessário.")

# Atualizar pip, setuptools e wheel
try:
    print("Atualizando pip, setuptools e wheel...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"])
    print("Atualização de pip, setuptools e wheel concluída com sucesso!")
except subprocess.CalledProcessError as e:
    print(f"Erro ao atualizar pip, setuptools e wheel. Detalhes: {e}")

# Verificar e instalar pacotes
for package in packages:
    try:
        # Tentar importar o pacote para verificar se já está instalado
        __import__(package.split("-")[0])  # Divide em casos de pacotes com nomes compostos
        print(f"{package} já está instalado.")
    except ImportError:
        print(f"{package} não está instalado. Instalando...")
        install_package(package)

# Mensagem final
print("\nVerificação de pacotes concluída.")


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from IPython.display import Image
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from functools import reduce
import pickle
import warnings
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist, pdist
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from ydata_profiling import ProfileReport
import os
from dateutil.relativedelta import relativedelta
import pyautogui
import math
from matplotlib.colors import LinearSegmentedColormap
from scipy.stats import norm
import statistics as st

Você está rodando em Python 3.9+

In [ ]:
print("Versão do Python:", sys.version)

Você está usando um ambiente virtual: Virtualenv ou Anaconda

In [ ]:
print("Ambiente virtual ativo:", sys.prefix)

Todas as bibliotecas usadas nesse exercícios estão instaladas em um ambiente virtual específico

Gere um arquivo de requerimentos (requirements.txt) com os pacotes necessários. É necessário se certificar que a versão do pacote está disponibilizada.

In [ ]:
# Gera o arquivo requirements.txt com os pacotes instalados
with open("requirements.txt", "w") as file:
    subprocess.run(["python", "-m", "pip", "freeze"], stdout=file)

print("Arquivo requirements.txt gerado com sucesso.")


Tire um printscreen do ambiente que será usado rodando em sua máquina.

In [ ]:
# Tira o print da tela e salva no arquivo screenshot.png
screenshot = pyautogui.screenshot()
screenshot.save("screenshot.png")

print("Screenshot salva como 'screenshot.png'")

Disponibilize os códigos gerados, assim como os artefatos acessórios (requirements.txt) e instruções em um repositório GIT público. (se isso não for feito, o diretório com esses arquivos deverá ser enviado compactado no moodle).

In [ ]:
print('link para o GITHUB: https://github.com/CBarrosoBRRJ/Algoritmos_de_Inteligencia_Artificial_para_Clusterizacao')

CONFIG

In [ ]:
# Ignorar avisos
warnings.filterwarnings('ignore')

# Configurações de exibição do Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Garantir que seaborn e matplotlib tenham gráficos amigáveis
# Configuração para visualizações
plt.style.use("default")
sns.set_theme(style="whitegrid")


### Escolha de base de dados


Escolha uma base de dados para realizar o trabalho. Essa base será usada em um problema de clusterização.

In [ ]:
df = pd.read_csv('DATA\DF_CLUSTER.csv', encoding='latin-1', on_bad_lines='skip', sep=';')

In [ ]:
# Exibindo informações gerais do dataset
print("\nInformações do dataset:")
print(df.info())

Escreva a justificativa para a escolha de dados, dando sua motivação e objetivos.

In [ ]:
print('Essa base é uma base real do meu trabalho, e a motivação é tentar aplicar o método em uma base que pode virar um projeto profissional.')

Mostre através de gráficos a faixa dinâmica das variáveis que serão usadas nas tarefas de clusterização. Analise os resultados mostrados. O que deve ser feito com os dados antes da etapa de clusterização?


In [ ]:
# Escolher as cores do heatmap (exemplo de gradiente personalizado)
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", ["#003f5c", "#FFF", "#ffa600"])

# Filtrar colunas categóricas relevantes
categorical_cols = [col for col in df.select_dtypes(include=['object', 'category']).columns]

# Garantir que existam colunas categóricas no DataFrame
if not categorical_cols:
    print("Não há colunas categóricas no DataFrame para análise.")
else:
    # Criar um DataFrame com contagens absolutas para as variáveis binárias
    count_data = df[categorical_cols].apply(lambda x: x.value_counts()).T.fillna(0)

    # Formatar os números absolutos com pontos de milhar
    count_data_formatted = count_data.applymap(lambda x: f"{int(x):,}".replace(",", "."))

    # Criar o DataFrame com percentuais
    percent_data = (count_data.div(count_data.sum(axis=1), axis=0) * 100).round(1)

    # Unificar contagens e percentuais
    heatmap_data = count_data_formatted + " (" + percent_data.astype(str) + "%)"

    # Criar o heatmap com valores absolutos e percentuais
    plt.figure(figsize=(15, 10))
    sns.heatmap(
        count_data,  # O heatmap utiliza os dados numéricos para as cores
        annot=heatmap_data.values,  # Adiciona a contagem e percentual como texto
        fmt="",  # Evita formatação padrão de números
        cmap=custom_cmap,
        linewidths=0.5,
        cbar_kws={"label": "Contagem"}  # Adiciona uma barra de cores
    )
    plt.title("Heatmap de Contagens e Percentuais de Variáveis Categóricas", fontsize=16, fontweight="bold")
    plt.xlabel("Categorias", fontsize=14)
    plt.ylabel("Variáveis", fontsize=14)
    plt.xticks(fontsize=12, rotation=0)
    plt.yticks(fontsize=15, rotation=0, font='arial')
    plt.show()

    # Explicação sobre como analisar
    print(
        "\nAnálise do Heatmap:\n"
        "1. As linhas representam as variáveis categóricas do dataset.\n"
        "2. As colunas mostram as categorias dentro de cada variável (ex.: 'Sim' e 'Não').\n"
        "3. Os valores no heatmap mostram a contagem absoluta (com pontos de milhar), seguidos do percentual em parênteses.\n"
        "4. Utilize este gráfico para identificar variáveis desbalanceadas ou categorias dominantes.\n"
    )


Realize o pré-processamento adequado dos dados. Descreva os passos necessários.

In [ ]:
print(
    "1. Entendimento do problema e dos dados\n"
    "2. Pré-processamento dos dados\n"
    "3. Redução de dimensionalidade (ex.: PCA)\n"
    "4. Definição de métricas de qualidade\n"
    "5. Escolha do número de clusters\n"
    "6. Aplicação de algoritmos de clusterização\n"
    "7. Avaliação e comparação de modelos\n"
    "8. Visualização dos clusters\n"
    "9. Interpretação e aplicação dos resultados\n"
)


### Clusterização

#### Pré-processamento dos dados

In [ ]:
# Transformando valores binários ('S', 'N') para numéricos (1, 0)
df_numeric = pd.get_dummies(df)
df_numeric.head(5)

In [ ]:
df_numeric.sum(axis=1).head()

In [ ]:
# Verificando as distancias entre os dados
X = df_numeric.div(df_numeric.sum(axis=1), axis='rows')
X.head(5)

#### Redução de dimensionalidade

In [ ]:
# Ajuste no PCA para suportar proporção de variância explicada
n_componentes = 0.9  # Alvo de 90% da variância explicada

In [ ]:
# Redução de dimensionalidade com PCA
pca = PCA(n_components=n_componentes)
pca_result = pca.fit_transform(X)

In [ ]:
# Determinar o número de componentes para 90% da variância explicada
explained_variance = pca.explained_variance_ratio_.cumsum()
components_90_variance = next((i + 1 for i, cumulative_variance in enumerate(explained_variance) if cumulative_variance >= n_componentes), None)
print(f"Componentes necessários para 90% da variância: {components_90_variance}")

In [ ]:
if components_90_variance is None:
    print("Nenhum número de componentes atinge 90% da variância explicada.")
    print(f"A maior variância explicada acumulada é {explained_variance[-1]:.2%}.")
else:
    print(f"O número de componentes principais necessários para explicar pelo menos 90% da variância é {components_90_variance}, que acumula aproximadamente {explained_variance[components_90_variance - 1]:.2%} da variância.")



In [ ]:
# Plotar variância explicada acumulada
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance) + 1), explained_variance, marker='o', linestyle='--', label='Cumulative Explained Variance')
plt.axhline(y=n_componentes, color='r', linestyle='--', label='90% Variance Threshold') # Rotulo

if components_90_variance is not None:
    plt.axvline(x=components_90_variance, color='g', linestyle='--', label=f'{components_90_variance} Components')

plt.title('Cumulative Explained Variance by PCA Components')
plt.xlabel('Number of PCA Components')
plt.ylabel('Cumulative Explained Variance')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Calculando a variância explicada pelos dois componentes principais
pca = PCA(n_components=2)
pca_2d = pca.fit_transform(X)
explained_variance_2d = pca.explained_variance_ratio_.sum()

# Visualizando PCA com 2 Componentes para interpretação gráfica
plt.figure(figsize=(10, 6))
plt.scatter(pca_2d[:, 0], pca_2d[:, 1], alpha=0.6, s=10, color='purple')
plt.title("PCA - Visualização com 2 Componentes Principais")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.grid(True)
plt.show()

# Informando o percentual de variância explicada
print(f"Os dois primeiros componentes principais explicam {explained_variance_2d:.2%} da variância total.")

#### Definição de métricas de qualidade

#### Escolha do número de clusters

In [ ]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.utils import resample  # Para amostragem


In [ ]:
# Otimização por amostragem
sample_size = 10000  # Defina o tamanho da amostra
pca_sample = resample(X, n_samples=min(sample_size, len(pca_result)), random_state=42)

In [ ]:
# Inicializar listas
num_clusters = list(range(2, 20))  # Ajuste o intervalo conforme necessário
soma_quadrados_entre_cluster = []
silhouette_scores = []

In [ ]:
# Soma total dos quadrados
soma_total = sum(pdist(pca_sample) ** 2) / pca_sample.shape[0]

In [ ]:
print("Iniciando análise do método do cotovelo com otimizações...\n")

In [ ]:
# Calcular a primeira e segunda derivada
inertia = soma_quadrados_entre_cluster  # Lista já existente no script
first_derivative = np.diff(inertia)
second_derivative = np.diff(first_derivative)

In [ ]:
for k in num_clusters:
    try:
        # Criar o modelo KMeans
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, init='k-means++')
        labels = kmeans.fit_predict(pca_sample)

        # Soma dos quadrados intra-cluster
        intra_cluster_sum = kmeans.inertia_
        soma_quadrados_entre_cluster.append(soma_total - intra_cluster_sum)

        # Calcular o Silhouette Score
        if k > 1:  # Apenas para k > 1
            sil_score = silhouette_score(pca_sample, labels, metric='euclidean')
            silhouette_scores.append(sil_score)

        # Feedback
        print(f"Clusters: {k}, Inércia: {intra_cluster_sum:.2f}, Silhueta: {sil_score:.4f}")
    except Exception as e:
        print(f"Erro ao calcular para k={k}: {e}")



In [ ]:
# Encontrar o índice do cotovelo
elbow_index = np.argmax(second_derivative) + 2  # Adicionar 2 para alinhar ao número de clusters
print(f"Melhor número de clusters pelo Método do Cotovelo: {elbow_index}")

In [ ]:
# Verificar preenchimento das listas
if not silhouette_scores or not soma_quadrados_entre_cluster:
    print("Erro: As listas de Silhouette Scores ou Soma dos Quadrados Entre Clusters estão vazias.")
else:
    # Determinar o melhor número de clusters com base no Silhouette Score
    best_silhouette_index = silhouette_scores.index(max(silhouette_scores))
    best_silhouette_cluster = num_clusters[best_silhouette_index]

    # Determinar o melhor número de clusters com base no Método do Cotovelo
    if len(soma_quadrados_entre_cluster) > 1:
        elbow_index = np.argmin(np.diff(soma_quadrados_entre_cluster))
        best_elbow_cluster = num_clusters[elbow_index + 1]
    else:
        best_elbow_cluster = None

    # Exibir os resultados
    print("\nResultados Finais:")
    print(f"Melhor número de clusters com base no Silhouette Score: {best_silhouette_cluster}")
    print(f"Melhor Silhouette Score: {max(silhouette_scores):.4f}")
    if best_elbow_cluster:
        print(f"Melhor número de clusters com base no Método do Cotovelo: {best_elbow_cluster}")
    else:
        print("Não foi possível determinar o melhor número de clusters pelo Método do Cotovelo.")


In [ ]:
print("\nCálculos concluídos.")

In [ ]:
# Plotar os dois gráficos lado a lado
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico 1: Variância Explicada
ax1.plot(num_clusters, soma_quadrados_entre_cluster[:len(num_clusters)] / soma_total * 100, 'b*-')
ax1.set_ylim((0, 100))
ax1.grid(True)
ax1.axhline(y=70, color='r', linestyle='--', label='70% Variância Explicada')
ax1.axhline(y=80, color='g', linestyle='--', label='80% Variância Explicada')
if best_elbow_cluster:
    ax1.axvline(x=best_elbow_cluster, color='purple', linestyle='--', label=f'Melhor k (Elbow) = {best_elbow_cluster}')
    ax1.set_xlabel('Número de Clusters')
    ax1.set_ylabel('Percentual de Variância Explicada')
    ax1.set_title('Variância Explicada x Valor de K')
    ax1.legend(loc='best')

# Gráfico 2: Silhouette Score
sns.lineplot(x=num_clusters, y=silhouette_scores[:len(num_clusters)], color='darkorange', marker='o', ax=ax2)
ax2.axhline(y=0.50, color='r', linestyle='--', label='Limite Inferior Aceitável (0.5)')
ax2.axhline(y=0.70, color='g', linestyle='--', label='Limite Superior Aceitável (0.7)')
ax2.axvline(x=best_silhouette_cluster, color='purple', linestyle='--', label=f'Melhor Cluster (Silhouette) = {best_silhouette_cluster}')
ax2.set_xlabel('Número de Clusters', size=12)
ax2.set_ylabel('Silhouette Score', size=12)
ax2.set_title('Silhouette Score por Número de Clusters', size=20)
ax2.legend(loc='best')

# Exibir os gráficos
plt.tight_layout()
plt.show()

In [ ]:
# Determinar o melhor número de clusters com base no Silhouette Score
best_k_silhouette = num_clusters[np.argmax(silhouette_scores)]
silhouette_silhouette = max(silhouette_scores)

# Determinar o melhor número de clusters com base no cotovelo (se aplicável)
if len(soma_quadrados_entre_cluster) > 1:
    elbow_index = np.argmin(np.diff(soma_quadrados_entre_cluster))
    best_num_clusters = num_clusters[elbow_index + 1] if elbow_index + 1 < len(num_clusters) else num_clusters[elbow_index]
    silhouette_elbow = silhouette_scores[num_clusters.index(best_num_clusters)]
else:
    best_num_clusters = None
    silhouette_elbow = None

# Exibir métricas para o melhor k (cotovelo), se aplicável
if silhouette_elbow is not None:
    print(f"\nMétricas para o melhor k (cotovelo): {best_num_clusters}")
    print(f" - Silhueta: {silhouette_elbow:.4f}")

# Exibir métricas para o melhor k com base na silhueta
print(f"\nMétricas para o melhor k (silhueta): {best_k_silhouette}")
print(f" - Silhueta: {silhouette_silhouette:.4f}")

# Avaliação da qualidade do cluster com base no Silhouette Score
if silhouette_silhouette >= 0.50:
    print("A qualidade do cluster para o melhor k (silhueta) é BOA.")
elif 0.25 <= silhouette_silhouette < 0.50:
    print("A qualidade do cluster para o melhor k (silhueta) é MODERADA.")
else:
    print("A qualidade do cluster para o melhor k (silhueta) é RUIM.")


#### Aplicação de algoritmos de clusterização

K-Means

In [ ]:
import matplotlib.cm as cm
from sklearn.metrics import silhouette_samples, silhouette_score
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

In [ ]:
# Número de clusters escolhidos para o exemplo
n_clusters = best_k_silhouette

In [ ]:
# Aplicando KMeans nos dados reduzidos pelo PCA
kmeans = KMeans(n_clusters=n_clusters, random_state=42, init='k-means++')

# Ajustando o KMeans com os dados transformados pelo PCA
y_pred_kmeans = kmeans.fit_predict(X)  # Gerando os rótulos dos clusters

# Agora você pode acessar os labels dos clusters
kmeans_labels = kmeans.labels_

kmeans_silhouette = silhouette_score(X, kmeans_labels)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

# Simulando dados para magnitude e cardinalidade
cluster_centers = kmeans.cluster_centers_
magnitudes = np.linalg.norm(cluster_centers, axis=1)  # Magnitude dos centróides
cardinalities = np.bincount(y_pred_kmeans)  # Cardinalidade de cada cluster

# Regressão linear para a linha de evolução
slope, intercept, r_value, p_value, std_err = linregress(cardinalities, magnitudes)
trend_line = slope * np.array(cardinalities) + intercept

# Criando uma figura com três gráficos lado a lado
plt.figure(figsize=(18, 5))

# Gráfico de Magnitude
plt.subplot(1, 3, 1)
plt.bar(range(len(magnitudes)), magnitudes)
plt.title('Gráfico de Magnitude')
plt.xlabel('Clusters')
plt.ylabel('Magnitude')
plt.xticks(range(len(magnitudes)))
plt.grid(axis='y')

# Gráfico de Cardinalidade
plt.subplot(1, 3, 2)
plt.bar(range(len(cardinalities)), cardinalities)
plt.title('Gráfico de Cardinalidade')
plt.xlabel('Clusters')
plt.ylabel('Cardinalidade')
plt.xticks(range(len(cardinalities)))
plt.grid(axis='y')

# Gráfico de Magnitude vs Cardinalidade
plt.subplot(1, 3, 3)
plt.scatter(cardinalities, magnitudes, c=range(len(cardinalities)), cmap='viridis', s=100, label='Clusters')
plt.plot(cardinalities, trend_line, color='red', linestyle='--', label='Linha de Evolução')
plt.title('Magnitude vs Cardinalidade')
plt.xlabel('Cardinalidade')
plt.ylabel('Magnitude')
plt.grid()
plt.legend()
for i in range(len(cardinalities)):
    plt.text(cardinalities[i], magnitudes[i], f'Cluster {i}', fontsize=9)

# Ajustando layout para que todos os gráficos estejam alinhados
plt.tight_layout()
plt.show()


In [ ]:
# Ajustar o tamanho da figura para exibir os três gráficos lado a lado
fig = plt.figure(figsize=(18, 6))

# Gráfico 1: Visualizando os clusters e os centróides no plano 2D usando PCA
ax1 = fig.add_subplot(131)
sc1 = ax1.scatter(pca_sample[:, 0], pca_sample[:, 1], c=kmeans.labels_, cmap='viridis', alpha=0.6)  # Colorir pelos clusters
ax1.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c='red', s=200, marker='X', label='Centroides')  # Adicionar os centróides
ax1.set_title("Clusters com base no PCA (2D) - Com Centróides")
ax1.set_xlabel("Componente Principal 1")
ax1.set_ylabel("Componente Principal 2")
fig.colorbar(sc1, ax=ax1, label="Clusters")
ax1.legend()

# Gráfico 2: Refazendo o PCA com 3 componentes para visualização 3D
pca_3d = PCA(n_components=3)
pca_3d_data = pca_3d.fit_transform(pca_sample)  # Usar os dados amostrados

# Gerando o gráfico 3D
ax2 = fig.add_subplot(132, projection='3d')
sc2 = ax2.scatter(pca_3d_data[:, 0], pca_3d_data[:, 1], pca_3d_data[:, 2], c=kmeans.labels_, cmap='viridis', alpha=0.6)  # Colorir pelos clusters
ax2.set_xlabel('Componente Principal 1')
ax2.set_ylabel('Componente Principal 2')
ax2.set_zlabel('Componente Principal 3')
ax2.set_title('Visualização 3D dos Clusters')
fig.colorbar(sc2, ax=ax2, label="Clusters")

# Gráfico 3: Silhouette
ax3 = fig.add_subplot(133)
y_lower = 10
sample_silhouette_values = silhouette_samples(pca_sample, kmeans.labels_)

# Configurar limites do gráfico de Silhouette
ax3.set_xlim([-0.1, 1])
ax3.set_ylim([0, len(pca_sample) + (kmeans.n_clusters + 1) * 10])

for i in range(kmeans.n_clusters):
    # Valores de silhueta para o cluster i
    ith_cluster_silhouette_values = sample_silhouette_values[kmeans.labels_ == i]
    ith_cluster_silhouette_values.sort()

    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = cm.nipy_spectral(float(i) / kmeans.n_clusters)
    ax3.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, facecolor=color, edgecolor=color, alpha=0.7)

    # Label dos clusters no meio
    ax3.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10  # Espaço entre clusters

# Linha vertical do valor médio do Silhouette Score
silhouette_avg = sample_silhouette_values.mean()
ax3.axvline(x=silhouette_avg, color="red", linestyle="--", label=f"Média: {silhouette_avg:.2f}")

ax3.set_title("Gráfico de Silhouette para os Clusters")
ax3.set_xlabel("Valores do coeficiente de silhouette")
ax3.legend()

# Ajustar layout
plt.tight_layout()
plt.show()



In [ ]:
print('Em construção')